In [1]:
import os
%pip install -q llama-index
%pip install -q llama-index-llms-groq
%pip install -q llama-index-embeddings-huggingface docx2txt
%pip install llama-index-llms-ollama

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached llama_index_core-0.12.5-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.12.5-py3-none-any.whl (1.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.10.68.post1
    Uninstalling llama-index-core-0.10.68.post1:
      Successfully uninstalled llama-index-core-0.10.68.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-program-openai 0.1.7 requires llama-index-core<0.11.0,>=0.10.57, but you have llama-index-core 0.12.5 which is incompatible.
llama-index-cli 0.1.13 requires llama-index-core<0.11.0,>=0.10.11.post1, but you have llama-index-core 0.12.5 which is incompatible.
llam

In [7]:
import nest_asyncio
nest_asyncio.apply()
import os
from dotenv import load_dotenv
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [9]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    SummaryIndex
)


from llama_index.core.settings import Settings

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

#LLM Config
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY)

# from llama_index.llms.openai import OpenAI
# 
# llm = OpenAI(
#     model="gpt-4o",
#     api_key=OPENAI_API_KEY,  # uses OPENAI_API_KEY env var by default
# )


embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512
from IPython.display import display, HTML

ValidationError: 2 validation errors for OpenAI
logprobs
  Field required [type=missing, input_value={'model': 'gpt-4o-mini', ...: None, 'strict': False}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
default_headers
  Input should be a valid dictionary [type=dict_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.9/v/dict_type

In [3]:
files = [file for file in os.listdir('./upload') if os.path.isfile(os.path.join('./upload', file))]

In [4]:
file_docs = {}
for file in files:
    file_docs[file] = SimpleDirectoryReader(
        input_files=[f"./upload/{file}"]
    ).load_data()


In [5]:
for file in files:
    print(file)

The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-Morgan-Kaufmann-2011.pdf
The Theory That Would Not Die How Bayes Rule Cracked the Enigma Code Hunted Down Russian Submarines and Emerged Triumphant from Two Centuries of Controversy by Sharon Bertsch McGrayne.pdf


In [6]:
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata

# Build agents dictionary
agents = {}

for file in files:
    # build vector index
    vector_index = VectorStoreIndex.from_documents(
        file_docs[file],
    )
    # build summary index
    summary_index = SummaryIndex.from_documents(
        file_docs[file],
    )
    # define query engines
    vector_query_engine = vector_index.as_query_engine()
    summary_query_engine = summary_index.as_query_engine()

    # define tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name="vector_tool",
                description=(
                    f"Useful for retrieving specific context from {file}"
                ),
            ),
        ),
        QueryEngineTool(
            query_engine=summary_query_engine,
            metadata=ToolMetadata(
                name="summary_tool",
                description=(
                    "Useful for summarization questions related to"
                    f" {file}"
                ),
            ),
        ),
    ]

    # build agent
    agent = ReActAgent.from_tools(
        query_engine_tools,
        llm=llm,
        verbose=True,
    )

    agents[file] = agent


In [1]:
from llama_index.core.schema import IndexNode

# define top-level nodes
objects = []
for file in files:
    # define index node that links to these agents
    wiki_summary = (
        f"This content contains the full book of {file}. Use this index if you need to lookup specific facts about {file}."
    )
    node = IndexNode(
        text=wiki_summary, index_id=file, obj=agents[file]
    )
    objects.append(node)


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /opt/anaconda3/envs/COS243/lib/python3.11/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


NameError: name 'files' is not defined

In [41]:
vector_index = VectorStoreIndex(
    objects=objects,
)
query_engine = vector_index.as_query_engine(similarity_top_k=1, verbose=True)


In [42]:
# Helper function for print
def print_response(response):
    display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

In [57]:
# should use Toronto agent -> vector tool
response = query_engine.query( "Provide a comprehensive overview of how Bayesian probability evolved from a theoretical concept to a critical wartime intelligence tool")


Retrieval entering The Theory That Would Not Die How Bayes Rule Cracked the Enigma Code, Hunted Down Russian Submarines, and Emerged Triumphant from Two Centuries of Controversy by Sharon Bertsch McGrayne.pdf: ReActAgent
Retrieving from object ReActAgent with query Provide a comprehensive overview of how Bayesian probability evolved from a theoretical concept to a critical wartime intelligence tool
> Running step 349de761-6eab-4189-8ba8-6871210a9877. Step input: Provide a comprehensive overview of how Bayesian probability evolved from a theoretical concept to a critical wartime intelligence tool
Thought: The user is asking for a comprehensive overview of the evolution of Bayesian probability, particularly its transition from a theoretical concept to a critical tool in wartime intelligence. I will need to gather detailed information from the book "The Theory That Would Not Die" by Sharon Bertsch McGrayne to provide an accurate response.
Action: summary_tool
Action Input: {'input': 'Over

TypeError: 'NoneType' object is not subscriptable

In [55]:
print(response.source_nodes.metadata)

[NodeWithScore(node=TextNode(id_='945126b6-7c8e-4322-ac1c-e03b7025ec8d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Bayesian probability originated as a theoretical framework aimed at updating beliefs based on new evidence. This iterative process involves three key components: Prior (the initial belief), Likelihood (the probability of observing the data given the belief), and Posterior (the updated belief after considering the new data). This method allows for continuous refinement of understanding as more information becomes available.\n\nOver time, Bayesian methods transitioned from purely theoretical applications to practical uses across various fields, notably in wartime intelligence. For example, the U.S. Army adopted Bayesian techniques for testing weapons systems, particularly in scenarios where large sample sizes were not feasible. This approach enabled analysts to merge test data with insights from simil

In [51]:
print(response)

Bayesian probability began as a theoretical framework focused on the iterative process of updating beliefs based on new information. This involved the concepts of Prior, Likelihood, and Posterior, which together allow for the continuous recalibration of beliefs as new data is introduced.

Over time, this theoretical foundation found practical applications, particularly in military contexts. The U.S. Army recognized the limitations of traditional frequentist methods, especially when dealing with large sample sizes, and adopted Bayesian techniques for testing weapons systems like the Stryker vehicles. These methods enabled the integration of test data with prior information and data from similar systems, proving effective in complex and dynamic environments.

Furthermore, Bayesian approaches have been instrumental in assessing terrorist threats. By combining expert opinions with historical data, these techniques facilitate the evaluation of potential risks, showcasing their adaptability 

In [52]:
import gradio as gr
with gr.Blocks() as demo:
    gr.Markdown("# 🧑‍💼 Local Librarian AI Chatbot")

    # File upload section
    with gr.Tab("File Upload"):
        gr.Markdown("## Upload Your Files to the System")
        file_upload = gr.File(label="Upload Documents", file_count="multiple", type="filepath")
        file_output = gr.Markdown(label="Uploaded Files")
        file_upload.upload(fn=handle_file_upload, inputs=file_upload, outputs=file_output)

    # Chat and search interface
    with gr.Tab("Search & Query"):
        gr.Markdown("## Ask the Librarian Anything")

        # Input fields for query and conversation history
        query_input = gr.Textbox(label="Search Query", placeholder="Enter your search query here...")
        search_button = gr.Button("Search")
        conversation_history = gr.Textbox(label="Conversation History", interactive=False, lines=10)
        source_display = gr.Textbox(label="Source Citations", interactive=False, lines=5)

        # Show example prompts
        prompt_examples = gr.Button("Show Prompt Examples")
        examples_output = gr.Markdown()

        search_button.click(fn=search_query,
                            inputs=[query_input, conversation_history, source_display, examples_output],
                            outputs=[conversation_history, source_display])
        prompt_examples.click(fn=show_prompt_examples, outputs=examples_output)

    # Instructions
    gr.Markdown("### How to use this app:")
    gr.Markdown("""
        1. In the 'File Upload' tab, upload your documents to make them available for search.
        2. In the 'Search & Query' tab, input your query in the 'Search Query' field.
        3. Press the 'Search' button to see the results and the source citations.
        4. You can review the conversation history as it evolves.
        5. Use the 'Show Prompt Examples' button to see some predefined prompts to guide you.
    """)

    with gr.Accordion("About this demo"):
        gr.Markdown(
            """
            This demo showcases several Gradio components for a local librarian AI chatbot:
            - File upload functionality to import documents.
            - Search functionality with conversation history and source citation.
            - Premade prompt examples to help users get started.
            - Simple Markdown and interactive components for a user-friendly experience.
            """
        )

demo.launch()


NameError: name 'handle_file_upload' is not defined